In [1]:
import os
import pandas as pd
from PIL import Image
import random
import wandb
import copy
import timm

import torch
import torch.nn as nn
import torch.optim as optm
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from sklearn.metrics import f1_score
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib import gridspec

from torchinfo import summary
from tqdm.auto import tqdm

%matplotlib inline
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cwd=os.path.dirname(os.getcwd())

In [2]:
#프로젝트 이름 설정과 저장경로
project_name='EfficientNet'
model_dir=os.path.join(os.getcwd(),f'saved/{project_name}/checkpoint')
result_dir=os.path.join(os.path.dirname(model_dir),'result')
wandb_dir=os.path.dirname(model_dir)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
if not os.path.isdir(result_dir):
    os.makedirs(result_dir)

In [3]:
#Randomness 제어
random_seed=2021
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)


In [4]:
transform={'trans':transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]),
'pretrans':transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
]),
'posttrans':transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])}

def make_images(meta,img_dir,train):
    images=[]
    labels=[]
    if train:
        for idx in range(len(meta)):
            folder_path=os.path.join(img_dir, meta.path[idx])
            for img in os.listdir(folder_path):
                if '._' in img:
                    continue
                images.append(os.path.join(folder_path,img))
                labels.append((('incorrect' in img)+('normal' in img)*2)*6+(meta.gender[idx]=='female')*3+(30<=meta.age[idx])+(60<=meta.age[idx]))
    else:
        for img_id in meta.ImageID:
            images.append(os.path.join(img_dir, img_id))
    return images,labels

class ImageDataset(Dataset):
    def __init__(self,transform=transform,train=True):
        self.train=train
        self.md=['info','train']
        self.path=[os.path.join(cwd,'input/data/eval'),os.path.join(cwd,'input/data/train')]
        self.meta=pd.read_csv(os.path.join(self.path[train], f'{self.md[train]}.csv'))
        self.img_dir=os.path.join(self.path[train],'images')
        self.classes=[('Wear','Incorrect','Not Wear'),('남','여'),('<30','>=30 and <60','>=60')]
        self.trans=transform['trans']
        
        self.images,self.labels=make_images(self.meta,self.img_dir,train)

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image=Image.open(self.images[idx])
        image=self.trans(image)
        if self.train:
            label=self.labels[idx]
        else:
            label=0
        return image,label

def make_images_mask_avg(meta,img_dir,train):
    images=[]
    labels=[]
    if train:
        for idx in range(len(meta)):
            folder_path=os.path.join(img_dir, meta.path[idx])
            for keyward in ['mask','incorrect_mask','normal']:
                file_list=[os.path.join(folder_path,img) for img in os.listdir(folder_path) if (keyward in img)and (len(img)-len(keyward)<=5)]
                images.append(file_list)
                labels.append((('incorrect'==keyward)+('normal'==keyward)*2)*6+(meta.gender[idx]=='female')*3+(30<=meta.age[idx])+(60<=meta.age[idx]))
    else:
        for img_id in meta.ImageID:
            images.append(os.path.join(img_dir, img_id))
    return images, labels

def img_avg(trans,paths):
    if len(paths)==1:
        return trans['pretrans'](Image.open(paths[0]))
    image=torch.zeros_like(trans['pretrans'](Image.open(paths[0])))
    for path in paths:
        image+=trans['pretrans'](Image.open(path))
    return image/len(paths)

class ImageAvgDataset(Dataset):
    def __init__(self,transform=transform,train=True):
        self.train=train
        self.md=['info','train']
        self.path=[os.path.join(cwd,'input/data/eval'),os.path.join(cwd,'input/data/train')]
        self.meta=pd.read_csv(os.path.join(self.path[train], f'{self.md[train]}.csv'))
        self.img_dir=os.path.join(self.path[train],'images')
        self.classes=[('Wear','Incorrect','Not Wear'),('남','여'),('<30','>=30 and <60','>=60')]
        self.trans=transform
        
        self.images,self.labels=make_images_mask_avg(self.meta,self.img_dir,train)

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        if self.train:
            label=self.labels[idx]
            image=img_avg(self.trans,self.images[idx])
            image=self.trans['posttrans'](image)
        else:
            image=Image.open(self.images[idx])
            image=self.trans['trans'](image)
            label=0
        return image,label


In [5]:
#사용할 부분 모델 불러오기
net_mask = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=3)
net_gender = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=2)
net_age = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=3)

In [6]:
#모델 구축
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.mask=net_mask
        self.gender=net_gender
        self.age=net_age
        
    def forward(self,x):
        mask=self.mask(x).view(x.size(0),3,1,1)
        gender=self.gender(x).view(x.size(0),1,2,1)
        age=self.age(x).view(x.size(0),1,1,3)
        return (mask*gender*age).view(x.size(0),-1)

In [7]:
#모델 설정하기

model=MyModel()
model.to(device)
optim=optm.Adam(model.parameters())
if os.listdir(model_dir):
    latest_model_id=sorted(os.listdir(model_dir))[-1]
    checkpoint=torch.load(os.path.join(model_dir,latest_model_id))
    model.load_state_dict(checkpoint['model_state_dict'])
    optim.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch=checkpoint['epoch']+1
    loss=checkpoint['loss']
else:
    epoch=1

In [8]:
#학습 설정하기
num_epochs=3
batch_size=50
learning_rate=0.001
weight_loss=True
config={'epochs':num_epochs,'batch_size':batch_size,'learning_rate':learning_rate}

weight=torch.tensor([1,1.2,4.5,0.8,0.7,4.5]+2*[5, 6.0, 22.5, 4.0, 3.5, 22.5]).to(device)
loss_fn_weight=nn.CrossEntropyLoss(weight=weight)
loss_fn=nn.CrossEntropyLoss()

In [9]:
model_weight=copy.deepcopy(model)
optim_weight=copy.deepcopy(optim)

In [ ]:
wandb.init(project=project_name,config=config,dir=wandb_dir)
for e in range(epoch,epoch+num_epochs):   
    for train in [False]:
        data=ImageDataset(train=train)

        if train:
            dataloader=DataLoader(data,batch_size=batch_size,shuffle=True)
            model.train()
        else:
            submission = pd.read_csv(os.path.join(data.path[0], 'info.csv'))
            dataloader=DataLoader(data,shuffle=False)
            model.eval()

        with tqdm(dataloader) as pbar:
            running_loss=0.
            running_acc=0.
            tot_pred=torch.tensor([]).to(device)
            tot_label=torch.tensor([]).to(device)
            for n,(image,label) in enumerate(pbar):
                image=image.to(device)
                label=label.to(device)

                logit=model(image)
                _,pred=torch.max(logit,1)

                if train:
                    optim.zero_grad()
                    loss=loss_fn(logit,label)
                    loss.backward()
                    optim.step()
                    running_loss+=loss.item()*image.size(0)
                    running_acc+=torch.sum(pred==label)
                    pbar.set_postfix({'epoch' : epoch, 'loss' : running_loss/(n+1), 'accuracy' : float(running_acc)/(n+1),'F1 score':f1_score(label.cpu(),pred.cpu(),average='macro')})

                tot_pred=torch.hstack((tot_pred,pred))
                tot_label=torch.hstack((tot_label,label))
                
            if train:
                epoch_loss=running_loss/len(dataloader.dataset)
                epoch_acc=running_acc/len(dataloader.dataset)
                wandb.log({'loss' : epoch_loss, 'accuracy' : epoch_acc,'f1 score':f1_score(tot_label.cpu(),tot_pred.cpu(),average='macro')})
                torch.save({'epoch':e,'loss':loss,'model_state_dict':model.state_dict(),'optimizer_state_dict':optim.state_dict()},f"{model_dir}checkpoint_{e}_{epoch_loss:.3f}_{epoch_acc:.3f}_{f1_score(tot_label.cpu(),tot_pred.cpu(),average='macro')}.pt")
            else:
                pass
                #submission['ans'] = tot_pred.cpu().numpy().astype('int64')
                #submission.to_csv(os.path.join(result_dir, f'submission_{project_name}_{e}.csv'), index=False)
wandb.close()

In [ ]:
wandb.init(project=project_name,config=config,dir=wandb_dir)
for e in range(epoch,epoch+num_epochs):   
    for train in [True]:
        data=ImageDataset(train=train)

        if train:
            dataloader=DataLoader(data,batch_size=batch_size,shuffle=True)
            model_weight.train()
        else:
            submission = pd.read_csv(os.path.join(data.path[0], 'info.csv'))
            dataloader=DataLoader(data,shuffle=False)
            model_weight.eval()

        with tqdm(dataloader) as pbar:
            running_loss=0.
            running_acc=0.
            tot_pred=torch.tensor([]).to(device)
            tot_label=torch.tensor([]).to(device)
            for n,(image,label) in enumerate(pbar):
                image=image.to(device)
                label=label.to(device)

                logit=model_weight(image)
                _,pred=torch.max(logit,1)

                if train:
                    optim_weight.zero_grad()
                    loss=loss_fn_weight(logit,label)
                    loss.backward()
                    optim_weight.step()
                    running_loss+=loss.item()*image.size(0)
                    running_acc+=torch.sum(pred==label)
                    pbar.set_postfix({'epoch' : epoch, 'loss' : running_loss/(n+1), 'accuracy' : float(running_acc)/(n+1),'F1 score':f1_score(label.cpu(),pred.cpu(),average='macro')})

                tot_pred=torch.hstack((tot_pred,pred))
                tot_label=torch.hstack((tot_label,label))
                
            if train:
                epoch_loss=running_loss/len(dataloader.dataset)
                epoch_acc=running_acc/len(dataloader.dataset)
                wandb.log({'loss' : epoch_loss, 'accuracy' : epoch_acc,'f1 score':f1_score(tot_label.cpu(),tot_pred.cpu(),average='macro')})
                #torch.save({'epoch':e,'loss':loss,'model_state_dict':model_weight.state_dict(),'optimizer_state_dict':optim_weight.state_dict()},f"{model_dir}checkpoint_{e}_{epoch_loss:.3f}_{epoch_acc:.3f}_{f1_score(tot_label.cpu(),tot_pred.cpu(),average='macro')}.pt")
            else:
                pass
                #submission['ans'] = tot_pred.cpu().numpy().astype('int64')
                #submission.to_csv(os.path.join(result_dir, f'submission_{project_name}_{e}.csv'), index=False)
wandb.close()